<a href="https://colab.research.google.com/github/Han1210/Material_Modeling_HX/blob/main/HW5/02_MLFF_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLFF (Machine Learning Force Field) Tutorial with CHGNet

This tutorial demonstrates how to use CHGNet for structure relaxation and energy/force prediction compared with DFT results.

## What is CHGNet?

CHGNet (Crystal Hamiltonian Graph neural Network) is a pretrained universal neural network potential that can predict:
- **Energy** (eV/atom)
- **Forces** (eV/Å)
- **Stress** (GPa)
- **Magnetic moments** (μB)

It was trained on over 1.5 million structures from the Materials Project database and provides near-DFT accuracy for atomistic simulations.


In [1]:
# Install required packages
%pip install chgnet
%pip install pymatgen
%pip install matplotlib
%pip install numpy

print("✓ All packages installed successfully!")


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 39.8 MB/s eta 0:00:00
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19172 sha256=77cd9a9d94ef9c18ab713844

In [2]:
# Import libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

# Import pymatgen for structure handling
from pymatgen.core import Structure
from pymatgen.io.vasp import Poscar

# Import CHGNet components
from chgnet.model.model import CHGNet
from chgnet.model import StructOptimizer

print("✓ All libraries imported successfully!")


✓ All libraries imported successfully!


In [3]:
# Load CHGNet model
print("Loading CHGNet model...")
chgnet = CHGNet.load()
print("✓ CHGNet model loaded successfully!")

Loading CHGNet model...
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
✓ CHGNet model loaded successfully!


In [11]:
# Check the current path
import os
print(os.getcwd())
# List the files in the current directory
!ls
# Clone the course repository from GitHub
!git clone https://github.com/hguangshuai/intro_ML_for_MSE/
# Change into the repository directory
%cd intro_ML_for_MSE

# Load one POSCAR file using pymatgen Structure.from_str() method
file_path = 'Data/poscar/HgORu_-5.41213.poscar'

print(f"Loading {file_path}...")

# Read POSCAR file content as string
with open(file_path, 'r') as f:
    poscar_str = f.read()

# Use pymatgen to read POSCAR from string
structure = Structure.from_str(poscar_str, fmt="poscar")

# Extract DFT energy from filename
dft_energy = -5.41213  # From filename

print(f"✓ Successfully loaded {file_path}")
print(f"Composition: {structure.composition}")
print(f"Formula: {structure.formula}")
print(f"Atoms: {len(structure)}")
print(f"DFT Energy: {dft_energy:.6f} eV/atom")
print(structure)

/content/intro_ML_for_MSE/intro_ML_for_MSE
01_Data_Overview_and_Loading.ipynb  Data     README.md	       Training
02_MLFF_example.ipynb		    LICENSE  requirements.txt  utils
Cloning into 'intro_ML_for_MSE'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 27 (delta 2), reused 27 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 775.65 KiB | 5.88 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE
Loading Data/poscar/HgORu_-5.41213.poscar...
✓ Successfully loaded Data/poscar/HgORu_-5.41213.poscar
Composition: Hg2 O4 Ru2
Formula: Hg2 Ru2 O4
Atoms: 8
DFT Energy: -5.412130 eV/atom
Full Formula (Hg2 Ru2 O4)
Reduced Formula: HgRuO2
abc   :   3.173243   5.307282   7.020744
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP       a     b      c
---  ----  ----  ----  -----


In [12]:
from chgnet.model import StructOptimizer

relaxer = StructOptimizer()
result = relaxer.relax(structure)
print("CHGNet relaxed structure", result["final_structure"])
print("relaxed total energy in eV:", result['trajectory'].energies[-1])

#get No. of atoms
print("No. of atoms:", len(structure))

#calculate energy per atom
print("Energy per atom:", result['trajectory'].energies[-1] / len(structure))

#calculate force magnitude
print("Force magnitude:", np.linalg.norm(result['trajectory'].forces[-1]))



CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
      Step     Time          Energy          fmax
FIRE:    0 20:14:06      -39.170090        2.696992
FIRE:    1 20:14:06      -39.529335        2.752327
FIRE:    2 20:14:06      -40.250393        2.828531
FIRE:    3 20:14:07      -41.341625        3.027887
FIRE:    4 20:14:07      -42.389240        2.530263
FIRE:    5 20:14:07      -43.163712        1.806183
FIRE:    6 20:14:07      -43.575527        0.934171
FIRE:    7 20:14:07      -43.546562        1.651884
FIRE:    8 20:14:07      -43.562698        1.625054
FIRE:    9 20:14:07      -43.592987        1.581914
FIRE:   10 20:14:08      -43.633671        1.508992
FIRE:   11 20:14:08      -43.679779        1.413945
FIRE:   12 20:14:08      -43.727436        1.307493
FIRE:   13 20:14:08      -43.773422        1.206355
FIRE:   14 20:14:08      -43.817581        1.079400
FIRE:   15 20:14:08      -43.860207        0.900354
FIRE:   16 20:14:08      -43.903610        0.

In [13]:
#AgAsO_-5.18793.poscar
# Check the current path
import os
print(os.getcwd())
# List the files in the current directory
!ls
# Clone the course repository from GitHub
!git clone https://github.com/hguangshuai/intro_ML_for_MSE/
# Change into the repository directory
%cd intro_ML_for_MSE

# Load one POSCAR file using pymatgen Structure.from_str() method
file_path = 'Data/poscar/AgAsO_-5.18793.poscar'

print(f"Loading {file_path}...")

# Read POSCAR file content as string
with open(file_path, 'r') as f:
    poscar_str = f.read()

# Use pymatgen to read POSCAR from string
structure = Structure.from_str(poscar_str, fmt="poscar")

# Extract DFT energy from filename
dft_energy = -5.18793  # From filename

print(f"✓ Successfully loaded {file_path}")
print(f"Composition: {structure.composition}")
print(f"Formula: {structure.formula}")
print(f"Atoms: {len(structure)}")
print(f"DFT Energy: {dft_energy:.6f} eV/atom")
print(structure)

/content/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE
01_Data_Overview_and_Loading.ipynb  Data     README.md	       Training
02_MLFF_example.ipynb		    LICENSE  requirements.txt  utils
Cloning into 'intro_ML_for_MSE'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 27 (delta 2), reused 27 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 775.65 KiB | 6.80 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE
Loading Data/poscar/AgAsO_-5.18793.poscar...
✓ Successfully loaded Data/poscar/AgAsO_-5.18793.poscar
Composition: Ag8 As8 O28
Formula: Ag8 As8 O28
Atoms: 44
DFT Energy: -5.187930 eV/atom
Full Formula (Ag8 As8 O28)
Reduced Formula: Ag2As2O7
abc   :   5.838056  12.561053   7.505991
angles:  90.000000 110.673000  90.000000
pbc   :       True       True       True
Sites (44)
  #  SP         a

In [14]:
#AgAsO_-5.18793.poscar
from chgnet.model import StructOptimizer

relaxer = StructOptimizer()
result = relaxer.relax(structure)
print("CHGNet relaxed structure", result["final_structure"])
print("relaxed total energy in eV:", result['trajectory'].energies[-1])

#get No. of atoms
print("No. of atoms:", len(structure))

#calculate energy per atom
print("Energy per atom:", result['trajectory'].energies[-1] / len(structure))

#calculate force magnitude
print("Force magnitude:", np.linalg.norm(result['trajectory'].forces[-1]))

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
      Step     Time          Energy          fmax
FIRE:    0 20:15:28     -242.287369        3.861103
FIRE:    1 20:15:29     -244.456024        1.554589
FIRE:    2 20:15:30     -244.875717        1.649984
FIRE:    3 20:15:30     -244.948181        1.473910
FIRE:    4 20:15:31     -245.068329        1.142814
FIRE:    5 20:15:32     -245.194916        0.707172
FIRE:    6 20:15:33     -245.297287        0.533618
FIRE:    7 20:15:34     -245.368637        0.678911
FIRE:    8 20:15:35     -245.427811        0.834872
FIRE:    9 20:15:36     -245.499924        0.811273
FIRE:   10 20:15:37     -245.597580        0.582062
FIRE:   11 20:15:38     -245.702011        0.431522
FIRE:   12 20:15:38     -245.780685        0.360454
FIRE:   13 20:15:39     -245.829208        0.497220
FIRE:   14 20:15:40     -245.877350        0.568948
FIRE:   15 20:15:41     -245.938934        0.493230
FIRE:   16 20:15:41     -245.969299        0.

In [15]:
##AlBaPt_-4.63365.poscar
# Check the current path
import os
print(os.getcwd())
# List the files in the current directory
!ls
# Clone the course repository from GitHub
!git clone https://github.com/hguangshuai/intro_ML_for_MSE/
# Change into the repository directory
%cd intro_ML_for_MSE

# Load one POSCAR file using pymatgen Structure.from_str() method
file_path = 'Data/poscar/AlBaPt_-4.63365.poscar'

print(f"Loading {file_path}...")

# Read POSCAR file content as string
with open(file_path, 'r') as f:
    poscar_str = f.read()

# Use pymatgen to read POSCAR from string
structure = Structure.from_str(poscar_str, fmt="poscar")

# Extract DFT energy from filename
dft_energy = -4.63365  # From filename

print(f"✓ Successfully loaded {file_path}")
print(f"Composition: {structure.composition}")
print(f"Formula: {structure.formula}")
print(f"Atoms: {len(structure)}")
print(f"DFT Energy: {dft_energy:.6f} eV/atom")
print(structure)

/content/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE
01_Data_Overview_and_Loading.ipynb  Data     README.md	       Training
02_MLFF_example.ipynb		    LICENSE  requirements.txt  utils
Cloning into 'intro_ML_for_MSE'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 27 (delta 2), reused 27 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 775.65 KiB | 5.75 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE
Loading Data/poscar/AlBaPt_-4.63365.poscar...
✓ Successfully loaded Data/poscar/AlBaPt_-4.63365.poscar
Composition: Al4 Ba2 Pt2
Formula: Ba2 Al4 Pt2
Atoms: 8
DFT Energy: -4.633650 eV/atom
Full Formula (Ba2 Al4 Pt2)
Reduced Formula: BaAl2Pt
abc   :   6.573299   6.573299   7.764699
angles:  90.000000  90.000000 138.481538
pbc   :       True       True      

In [16]:
#AlBaPt_-4.63365.poscar
from chgnet.model import StructOptimizer

relaxer = StructOptimizer()
result = relaxer.relax(structure)
print("CHGNet relaxed structure", result["final_structure"])
print("relaxed total energy in eV:", result['trajectory'].energies[-1])

#get No. of atoms
print("No. of atoms:", len(structure))

#calculate energy per atom
print("Energy per atom:", result['trajectory'].energies[-1] / len(structure))

#calculate force magnitude
print("Force magnitude:", np.linalg.norm(result['trajectory'].forces[-1]))

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
      Step     Time          Energy          fmax
FIRE:    0 20:16:59      -35.074272        1.517277
FIRE:    1 20:16:59      -35.186314        1.511468
FIRE:    2 20:16:59      -35.404305        1.471325
FIRE:    3 20:16:59      -35.713409        1.417110
FIRE:    4 20:17:00      -36.092987        1.277097
FIRE:    5 20:17:00      -36.487041        1.099470
FIRE:    6 20:17:00      -36.840836        0.809169
FIRE:    7 20:17:00      -37.070797        0.400607
FIRE:    8 20:17:00      -37.123077        0.417202
FIRE:    9 20:17:00      -37.124779        0.413919
FIRE:   10 20:17:01      -37.128120        0.407416
FIRE:   11 20:17:01      -37.132954        0.397699
FIRE:   12 20:17:01      -37.139088        0.384557
FIRE:   13 20:17:01      -37.146297        0.367723
FIRE:   14 20:17:01      -37.154278        0.347256
FIRE:   15 20:17:01      -37.162678        0.323327
FIRE:   16 20:17:01      -37.171997        0.

In [17]:
#Data/poscar/BaFPd_-5.28347.poscar
# Check the current path
import os
print(os.getcwd())
# List the files in the current directory
!ls
# Clone the course repository from GitHub
!git clone https://github.com/hguangshuai/intro_ML_for_MSE/
# Change into the repository directory
%cd intro_ML_for_MSE

# Load one POSCAR file using pymatgen Structure.from_str() method
file_path = 'Data/poscar/BaFPd_-5.28347.poscar'

print(f"Loading {file_path}...")

# Read POSCAR file content as string
with open(file_path, 'r') as f:
    poscar_str = f.read()

# Use pymatgen to read POSCAR from string
structure = Structure.from_str(poscar_str, fmt="poscar")

# Extract DFT energy from filename
dft_energy = -5.28347  # From filename

print(f"✓ Successfully loaded {file_path}")
print(f"Composition: {structure.composition}")
print(f"Formula: {structure.formula}")
print(f"Atoms: {len(structure)}")
print(f"DFT Energy: {dft_energy:.6f} eV/atom")
print(structure)

/content/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE
01_Data_Overview_and_Loading.ipynb  Data     README.md	       Training
02_MLFF_example.ipynb		    LICENSE  requirements.txt  utils
Cloning into 'intro_ML_for_MSE'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 27 (delta 2), reused 27 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 775.65 KiB | 5.88 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE
Loading Data/poscar/BaFPd_-5.28347.poscar...
✓ Successfully loaded Data/poscar/BaFPd_-5.28347.poscar
Composition: Ba4 F12 Pd2
Formula: Ba4 Pd2 F12
Atoms: 18
DFT Energy: -5.283470 eV/atom
Full Formula (Ba4 Pd2 F12)
Reduced Formula: Ba2PdF6
abc   :   9.753562   9.753562   6.612149
angles:  90.000000  90.000000  39.660661
pb

In [18]:
#Data/poscar/BaFPd_-5.28347.poscar
from chgnet.model import StructOptimizer

relaxer = StructOptimizer()
result = relaxer.relax(structure)
print("CHGNet relaxed structure", result["final_structure"])
print("relaxed total energy in eV:", result['trajectory'].energies[-1])

#get No. of atoms
print("No. of atoms:", len(structure))

#calculate energy per atom
print("Energy per atom:", result['trajectory'].energies[-1] / len(structure))

#calculate force magnitude
print("Force magnitude:", np.linalg.norm(result['trajectory'].forces[-1]))

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
      Step     Time          Energy          fmax
FIRE:    0 20:18:26      -98.712044        1.051783
FIRE:    1 20:18:26      -98.777634        1.033363
FIRE:    2 20:18:26      -98.897354        0.981442
FIRE:    3 20:18:26      -99.037872        0.868741
FIRE:    4 20:18:26      -99.160782        0.769311
FIRE:    5 20:18:26      -99.242462        0.614816
FIRE:    6 20:18:27      -99.286423        0.525109
FIRE:    7 20:18:27      -99.337189        0.671164
FIRE:    8 20:18:27      -99.445740        0.655468
FIRE:    9 20:18:27      -99.610535        0.447892
FIRE:   10 20:18:27      -99.763756        0.434837
FIRE:   11 20:18:28      -99.841064        0.500071
FIRE:   12 20:18:28      -99.894928        0.449434
FIRE:   13 20:18:28      -99.988693        0.378868
FIRE:   14 20:18:28     -100.060928        0.069986
CHGNet relaxed structure Full Formula (Ba4 Pd2 F12)
Reduced Formula: Ba2PdF6
abc   :   9.262409  

In [19]:
#Data/poscar/FeLaO_-8.04788.poscar
# Check the current path
import os
print(os.getcwd())
# List the files in the current directory
!ls
# Clone the course repository from GitHub
!git clone https://github.com/hguangshuai/intro_ML_for_MSE/
# Change into the repository directory
%cd intro_ML_for_MSE

# Load one POSCAR file using pymatgen Structure.from_str() method
file_path = 'Data/poscar/FeLaO_-8.04788.poscar'

print(f"Loading {file_path}...")

# Read POSCAR file content as string
with open(file_path, 'r') as f:
    poscar_str = f.read()

# Use pymatgen to read POSCAR from string
structure = Structure.from_str(poscar_str, fmt="poscar")

# Extract DFT energy from filename
dft_energy = -8.04788  # From filename

print(f"✓ Successfully loaded {file_path}")
print(f"Composition: {structure.composition}")
print(f"Formula: {structure.formula}")
print(f"Atoms: {len(structure)}")
print(f"DFT Energy: {dft_energy:.6f} eV/atom")
print(structure)

/content/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE
01_Data_Overview_and_Loading.ipynb  Data     README.md	       Training
02_MLFF_example.ipynb		    LICENSE  requirements.txt  utils
Cloning into 'intro_ML_for_MSE'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 27 (delta 2), reused 27 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 775.65 KiB | 6.86 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE
Loading Data/poscar/FeLaO_-8.04788.poscar...
✓ Successfully loaded Data/poscar/FeLaO_-8.04788.poscar
Composition: Fe4 La4 O12
Formula: La4 Fe4 O12
Atoms: 20
DFT Energy: -8.047880 eV/atom
Full Formula (La4 Fe4 O12)
Reduced Formula: LaFeO3
abc   :   5.858549   8.269896   5.849390
angles:  9

In [20]:
#Data/poscar/FeLaO_-8.04788.poscar
from chgnet.model import StructOptimizer

relaxer = StructOptimizer()
result = relaxer.relax(structure)
print("CHGNet relaxed structure", result["final_structure"])
print("relaxed total energy in eV:", result['trajectory'].energies[-1])

#get No. of atoms
print("No. of atoms:", len(structure))

#calculate energy per atom
print("Energy per atom:", result['trajectory'].energies[-1] / len(structure))

#calculate force magnitude
print("Force magnitude:", np.linalg.norm(result['trajectory'].forces[-1]))

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
      Step     Time          Energy          fmax
FIRE:    0 20:19:38     -166.566345        1.528825
FIRE:    1 20:19:38     -166.657715        1.496870
FIRE:    2 20:19:38     -166.833466        1.437479
FIRE:    3 20:19:39     -167.079865        1.348056
FIRE:    4 20:19:39     -167.375702        1.235291
FIRE:    5 20:19:39     -167.695328        1.090916
FIRE:    6 20:19:40     -168.003265        0.910372
FIRE:    7 20:19:40     -168.266281        0.715123
FIRE:    8 20:19:40     -168.465805        0.513849
FIRE:    9 20:19:41     -168.573013        0.361665
FIRE:   10 20:19:41     -168.612915        0.310812
FIRE:   11 20:19:41     -168.613556        0.360357
FIRE:   12 20:19:42     -168.620316        0.357154
FIRE:   13 20:19:42     -168.633270        0.350821
FIRE:   14 20:19:43     -168.651291        0.341733
FIRE:   15 20:19:43     -168.672852        0.330809
FIRE:   16 20:19:44     -168.696594        0.

In [21]:
# Data/poscar/LiOSi_-7.0131.poscar
# Check the current path
import os
print(os.getcwd())
# List the files in the current directory
!ls
# Clone the course repository from GitHub
!git clone https://github.com/hguangshuai/intro_ML_for_MSE/
# Change into the repository directory
%cd intro_ML_for_MSE

# Load one POSCAR file using pymatgen Structure.from_str() method
file_path = 'Data/poscar/LiOSi_-7.0131.poscar'

print(f"Loading {file_path}...")

# Read POSCAR file content as string
with open(file_path, 'r') as f:
    poscar_str = f.read()

# Use pymatgen to read POSCAR from string
structure = Structure.from_str(poscar_str, fmt="poscar")

# Extract DFT energy from filename
dft_energy = -7.0131  # From filename

print(f"✓ Successfully loaded {file_path}")
print(f"Composition: {structure.composition}")
print(f"Formula: {structure.formula}")
print(f"Atoms: {len(structure)}")
print(f"DFT Energy: {dft_energy:.6f} eV/atom")
print(structure)

/content/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE
01_Data_Overview_and_Loading.ipynb  Data     README.md	       Training
02_MLFF_example.ipynb		    LICENSE  requirements.txt  utils
Cloning into 'intro_ML_for_MSE'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 27 (delta 2), reused 27 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 775.65 KiB | 6.06 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE/intro_ML_for_MSE
Loading Data/poscar/LiOSi_-7.0131.poscar...
✓ Successfully loaded Data/poscar/LiOSi_-7.0131.poscar
Composition: Li4 O10 Si4
Formula: Li4 Si4 O10
Atoms: 18
DFT Energy: -7.013100 eV/atom
Full Formula (Li4 Si4 O10)
Reduced Formula: Li2Si2O5
abc   :   8.03619

In [22]:
# Data/poscar/LiOSi_-7.0131.poscar
from chgnet.model import StructOptimizer

relaxer = StructOptimizer()
result = relaxer.relax(structure)
print("CHGNet relaxed structure", result["final_structure"])
print("relaxed total energy in eV:", result['trajectory'].energies[-1])

#get No. of atoms
print("No. of atoms:", len(structure))

#calculate energy per atom
print("Energy per atom:", result['trajectory'].energies[-1] / len(structure))

#calculate force magnitude
print("Force magnitude:", np.linalg.norm(result['trajectory'].forces[-1]))

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
      Step     Time          Energy          fmax
FIRE:    0 20:20:43     -131.799789        2.387987
FIRE:    1 20:20:43     -131.997681        1.126897
FIRE:    2 20:20:44     -132.133499        1.016301
FIRE:    3 20:20:44     -132.184219        1.959424
FIRE:    4 20:20:44     -132.218567        1.595800
FIRE:    5 20:20:45     -132.266281        0.977072
FIRE:    6 20:20:45     -132.301056        0.417276
FIRE:    7 20:20:45     -132.311249        0.658353
FIRE:    8 20:20:46     -132.312653        0.635314
FIRE:    9 20:20:46     -132.315308        0.590779
FIRE:   10 20:20:47     -132.318970        0.527862
FIRE:   11 20:20:47     -132.323303        0.451240
FIRE:   12 20:20:47     -132.328003        0.370002
FIRE:   13 20:20:48     -132.332886        0.360648
FIRE:   14 20:20:48     -132.337708        0.350043
FIRE:   15 20:20:48     -132.342941        0.336167
FIRE:   16 20:20:48     -132.348740        0.